# ccClub-教育平台專案 網路爬蟲（多頁）



##### Copyright

Licensed under the MIT License

In [ ]:
# MIT License

# Copyright (c) 2020 Lu Jui-Ting

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

# GC 環境設定
在 Google Colab 上才需要跑這個部分

In [ ]:
from IPython import display # Some of the output are just too long and too ugly so need this to adjust the display 
!pip install -q gwpy # To omit output, add "%%capture" in the beginning of the cell

     |████████████████████████████████| 1.4MB 5.3MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 61kB 5.2MB/s 
     |████████████████████████████████| 2.6MB 16.5MB/s 


Set up Chrome driver for selenium. This might take a while, so be patient.

In [ ]:
%%capture 
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

## Mount Google Drive

Follow the authentification and enter the key.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


When GD is succesfully mounted, the output will show
```
Mounted at /content/gdrive
```
You will also find **gdrive** if you click on the third icon on the left like in this picture.
![](https://i.imgur.com/C7JazX7.png)

# 其他設定

- 載入套件
- Selenium Driver 設定

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
import pandas as pd 
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
mydriver = webdriver.Chrome('chromedriver', options=chrome_options)
page = 0
df = pd.DataFrame({"標題":[], "url": [], "星數" : [], "日期" : [], "評論標題" : [], "評論內文" : []})

# Define total page
first_url = 'https://hahow.in/courses?search=python'
def findTotalPage(myurl):
    mydriver2 = webdriver.Chrome('chromedriver', options=chrome_options)
    first_page = mydriver2.get(myurl)
    time.sleep(3) 
    first_soup = BeautifulSoup(mydriver2.page_source, 'html.parser')
    page_block = first_soup.find_all('ul',{'class':"rc-pagination gbga9a-0 jYLVph"})
    assert len(page_block)==1
    pages_plus_arrows = page_block[0].find_all('li') # 這邊會多算兩個箭頭
    return len(pages_plus_arrows) - 2
total_page = findTotalPage(first_url)

while (page < total_page):
    page += 1
    url = 'https://hahow.in/courses?search=python&page='+str(page)
    gettingurl = mydriver.get(url)
    print("資料爬起來! >>> 第"+str(page)+"/"+str(total_page)+"頁：")
    time.sleep(3)
    source = mydriver.page_source
    soup = BeautifulSoup(source, 'html.parser')
    #print(soup.prettify())
    class_block = soup.find_all('div',{'class':"sc-10r5mg2-0 fVNHJD hh-course-brief relative block "}) # be ware that there is a space at the end
    #print(class_block)
    for class_ in class_block :
        #print(class_)
        class_href = class_.find('div','cover-wrap relative').find('a')
        class_url ='https://hahow.in'+ class_href.get('href')
        class_img = class_.find('div', {'class':'cover-image-wrap relative'}).find('img')
        #print(class_.find('div', {'class':'cover-image-wrap relative'}))
        if class_img!= None and class_img.has_attr('src'):
          print(class_img['src'])
        # if class_img!= None and class_img.has_attr('alt'): # another way to find title
        #  print(class_img['alt'])
        response = mydriver.get(class_url)
        time.sleep(3)
        response_source = mydriver.page_source
        response_soup= BeautifulSoup(response_source, 'html.parser')
        #title = response_soup.find('h1',{'class':'title text-center'}).text
        title = response_soup.find('h1').text
        print(title)
        #print(class_img)
        #print(class_url)
        if 'Python' in title :
            seeMoreButton = mydriver.find_elements_by_xpath("//button[@class='sc-1a6j6ze-0 cYdxxq b21euj-2 gMMXlv']")
            if len(seeMoreButton) != 0 :
                press = seeMoreButton[0]
                press.click()
            comments= response_soup.find_all('div',class_='wei2cc-1 gUylJK marg-b-25')

            # Initialisation
            count = 0
            stars = []; dates = []; shortTitles = []; longComments = []
            for comment in comments :
                #print('-----------'+ str(count)+ '-----------')                
                rating = comment.find('div',{'class':'star-ratings'})
                star = rating.attrs['title']
                date = comment.find('time').text
                shortTitle = comment.find('p',{'class':'text-strong marg-b-5'}).text
                longComment = comment.find('p',class_='marg-b-0').text
                #print(stars, course_time, shortTitle)
                #print(longComment.strip())
                stars.append(star)
                dates.append(date)
                shortTitles.append(shortTitle)
                longComments.append(longComment)
                count += 1
            if count!=0:
                df_of_1_course = pd.DataFrame({"標題": title, \
                                   "url": class_url, \
                                   "星數" : stars, \
                                   "日期" : dates, \
                                   "評論標題" : shortTitles, \
                                   "評論內文" : longComments} )
                # update the dataframe
                df = df.append(df_of_1_course, ignore_index = True) 
            print('--------收錄'+ str(count)+ '個評論-----------')  
    



資料爬起來! >>> 第1/2頁：
https://images.api.hahow.in/images/5f9fd2dff506d369edfe8c2d?width=300
Python 行銷資料科學 ：基礎語法入門最前線
--------收錄0個評論-----------
https://images.api.hahow.in/images/5f8d584e11eb8d43e01ce3e1?width=300
Python 資料分析：AI 機器學習入門到應用
--------收錄0個評論-----------
https://images.api.hahow.in/images/5f26b5b9dc9e1b518a646ee6?width=300
NBA 戰力分析！簡單打造 Python 數位報表
--------收錄0個評論-----------
Python 進階課程：觀念、語法、專案、爬蟲
--------收錄0個評論-----------
顧客分類大師：Python x RFM 會員經營新觀點
--------收錄3個評論-----------
打造動態報表！雲端 Python 爬蟲資料流
--------收錄3個評論-----------
Python 0 到 1 基礎商業數據分析實戰
--------收錄3個評論-----------
Python 資料分析 - 網頁爬蟲與視覺呈現
--------收錄3個評論-----------
自學程式如何快速入門 - 以 Python/JS 為例
--------收錄3個評論-----------
AI 行銷學：用 Python 機器學習創造商業新價值
--------收錄3個評論-----------
用 Python 打造自己的股票小秘書
--------收錄3個評論-----------
Python 入門特訓 - 基礎實作到證照攻略
--------收錄3個評論-----------
用 Python 理財：打造自己的 AI 股票理專
--------收錄3個評論-----------
用 Python 理財：打造加密貨幣實戰策略
--------收錄3個評論-----------
用 Python 理財：打造小資族選股策略
--------收錄3個評論-----------
Python 資料分析＆

In [ ]:
df

,標題,url,星數,日期,評論標題,評論內文
0,顧客分類大師：Python x RFM 會員經營新觀點,https://hahow.in/courses/5e7883d29c4b140023a79c51,5 Stars,2020.08.03,python與RFM的快速媒合課程,上完後就可以理解在python中使用RFM的邏輯\n老師在課程裡已寫好基礎的四大面向RFM視...
1,顧客分類大師：Python x RFM 會員經營新觀點,https://hahow.in/courses/5e7883d29c4b140023a79c51,5 Stars,2 天前,內容豐富,除了如何使用軟體操作外，也提供不同面向圖表的判別意涵，就算這堂課上完後，在程式部分沒有跟上，...
2,顧客分類大師：Python x RFM 會員經營新觀點,https://hahow.in/courses/5e7883d29c4b140023a79c51,5 Stars,2020.09.21,實用的超值課程,教學細心清晰，循序漸進~~ \n活化大腦，提升思考，很有收穫！
3,打造動態報表！雲端 Python 爬蟲資料流,https://hahow.in/courses/5e64bb261c966500239069f9,5 Stars,2020.10.08,感覺挺好的,1. 每個步驟都講得挺清楚的，過程中不需要自己debug。看完課程後，的確有能力操作自己的資...
4,打造動態報表！雲端 Python 爬蟲資料流,https://hahow.in/courses/5e64bb261c966500239069f9,5 Stars,2020.10.26,推薦,學了Python一段時間了\n一直想用爬蟲及報表自動化\n發現老師開了這堂動態報表非常吸引我...
5,打造動態報表！雲端 Python 爬蟲資料流,https://hahow.in/courses/5e64bb261c966500239069f9,5 Stars,2020.10.09,十分推薦的課程,之前在網路上學習了很多python爬蟲程式，但是一直不知道如何去應用，每次爬到的資料都得自己...
6,Python 0 到 1 基礎商業數據分析實戰,https://hahow.in/courses/5e3d127f5b02f600245861d9,5 Stars,2020.05.29,非常實用的課程,Ivan老師講得非常清楚，課程也循序漸進，\n我之前並沒有學過任何語言程式，但上完這堂課後，...
7,Python 0 到 1 基礎商業數據分析實戰,https://hahow.in/courses/5e3d127f5b02f600245861d9,5 Stars,2020.05.22,希望老師在解釋程式碼時能在說明詳細一點,老師的課程十分實用，不是很死板的python程式碼課程，可以很快速將老師寫好的程式碼直接應用...
8,Python 0 到 1 基礎商業數據分析實戰,https://hahow.in/courses/5e3d127f5b02f600245861d9,5 Stars,2020.06.07,可以馬上應用到工作中，非常實用的課程,Ivan老師的課程以貼近商業的範例來發展課程，可以很快應用到工作上的分析工作。\n有成果，就...
9,Python 資料分析 - 網頁爬蟲與視覺呈現,https://hahow.in/courses/5d9c405e6f4b130023e9999d,5 Stars,2020.10.11,值得首選,"首次上Pyhotn網頁擷取與資料,內容就會一頭霧水就很像走進了小森林一樣,腦袋會很混亂、心裡..."


In [ ]:
import datetime
current_date = datetime.datetime.today().strftime ('%d-%b-%Y')
df.to_csv(r'/content/gdrive/My Drive/ccClub-教育平台專案/'+str(page)+'page_result_'+str(current_date)+'.csv')